In [317]:
import pandas as pnd
import heapq
from collections import Counter

iris = pnd.read_csv('iris.csv')
#source: https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv

In [318]:
import numpy as np

data_train = np.array(iris.iloc[:, 0:4])

In [319]:
DB = data_train
DB_len = len(DB)
label = [None]*DB_len
eps = 2.35
minPts = 5

In [320]:
def distFunc(Q, P):
    return np.linalg.norm(DB[Q]-DB[P])

In [321]:
def RangeQuery(i):
    Neighbors = []
    for j in range(DB_len):
        if distFunc(i, j) <= eps:
            Neighbors.append(j)
    return Neighbors

In [322]:
def DFS(S,j,C):
    if label[S[j]] == -1: label[S[j]] = C
    if label[S[j]] == None:
        label[S[j]] = C
        N = RangeQuery(S[j])
        if len(N) >= minPts:
            for i in range(len(N)):
                S.append(N[i])
        if len(S) != j+1:
            DFS(S,j+1,C)

In [323]:
def DBSCAN():
    C = 0
    for i in range(DB_len):
        if label[i] != None: continue
        N = RangeQuery(i)
        if len(N) < minPts:
            label[i] = -1
            continue

        C = C + 1
        label[i] = C
        S = [x for x in N if not x == i]
        if len(S) != 0:
            DFS(S,0,C)

In [324]:
DBSCAN()

outlier = Counter(label)[-1]
cluster = len([x for x in list(set(label)) if x != -1])

print "Cluster number: " + str(cluster)
print "Outlier number: " + str(outlier)
print "Labels:"
print label

Cluster number: 5
Outlier number: 0
Labels:
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
